In [1]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse



# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()


# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package libfuse2:amd64.
(Reading database ... 22298 files and directories currently installed.)
Preparing to unpack .../libfuse2_2.9.7-1ubuntu1_amd64.deb ...
Unpacking libfuse2:amd64 (2.9.7-1ubuntu1) ...
Selecting previously unselected package fuse.
Preparing to unpack .../fuse_2.9.7-1ubuntu1_amd64.deb ...
Unpacking fuse (2.9.7-1ubuntu1) ...
Selecting previously unselected package google-drive-ocamlfuse.
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu2~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu2~ubuntu18.04.1) ...
Setting up libfuse2:amd64 (2.9.7-1ubuntu1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
Setting up fuse (2.9.7-1ubuntu1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu2~ubuntu18.04.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleus

In [0]:
!mkdir -p My Drive
!google-drive-ocamlfuse -o nonempty My Drive

In [0]:
import os
os.chdir('Drive/ML_Project')
# !cd Drive/Colab\ Notebooks/vgg_tinyimagenet

In [6]:
!pip3 install --no-cache-dir -I pillow

    100% |████████████████████████████████| 2.0MB 63.2MB/s 


In [7]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision
import torch
torch.__version__

tcmalloc: large alloc 1073750016 bytes == 0x5b66e000 @  0x7ff998f1c2a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641


'0.4.0'

In [8]:
! nvidia-smi

Thu Nov 29 04:25:26 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 396.44                 Driver Version: 396.44                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [9]:
!pip3 install tensorboardX

    100% |████████████████████████████████| 71kB 5.3MB/s 


In [0]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
import torch.nn.functional as F
import matplotlib.pyplot as plt
import math
import time
import os
import copy
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
from PIL import Image
# from FCN8 import *
from segnet import segnet
from createDataset import MyDataset
from utils import *
from torchvision.utils import make_grid, save_image
import datetime
from sklearn.model_selection import train_test_split

from tensorboardX import SummaryWriter
import cv2

In [0]:
LOG_DIR = './logs/'


now = str(datetime.datetime.now())
OUTPUTS_DIR = './outputs/'

if not os.path.exists(LOG_DIR):
	os.makedirs(LOG_DIR)

if not os.path.exists(OUTPUTS_DIR):
	os.makedirs(OUTPUTS_DIR)
OUTPUTS_DIR = OUTPUTS_DIR + now + '/'
if not os.path.exists(OUTPUTS_DIR):
	os.makedirs(OUTPUTS_DIR)
if not os.path.exists(LOG_DIR+now):
	os.makedirs(LOG_DIR+now)

summary_writer = SummaryWriter(LOG_DIR + now)

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device('cpu')
print(device)
max_epoch = 10
labels = 11          #unused
dataroot = 'data/'
batch = 16
save_after = 1
lr = 1.0e-3
num_classes=39
img_size = (128,128)
momentum = 0.9
weight_decay = 1.0e-4
resume = True
log = "error_log.txt"


cuda


In [13]:
########### Model ###########
# # model = FCN8s(labels).to(device)
# save_file = 'weights_fcn.pth'
# model = AutoEncoder(labels).to(device)
# model = nn.DataParallel(model)
# if(resume):
#     model.load_state_dict(torch.load(save_file))
#     f = open(log, "a")
# else:
#     f = open(log, "w")




############ Segnet ####################
save_file = 'weights_segnet.pth'
vgg16 = models.vgg16_bn(pretrained=True)
model = segnet(num_classes)
model.init_vgg16_params(vgg16)
model = model.to(device)
if(resume):
    
    model.load_state_dict(torch.load(save_file))
    print('yes')
    f = open(log, "a")
else:
    f = open(log, "w")


Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.torch/models/vgg16_bn-6c64b313.pth
100%|██████████| 553507836/553507836 [00:05<00:00, 95955299.50it/s] 


yes


In [0]:
########### Transforms ###########
mean_std = ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
input_transforms = transforms.Compose([
        transforms.Resize(img_size, interpolation = 1),
        transforms.ToTensor(),
])
to_tensor =  transforms.Compose([transforms.ToTensor()])


In [0]:
########### Dataloader ###########
seg_path = 'gt_labels/'
img_path = 'leftImg8bit_orig/'

colpath = os.path.join(dataroot, img_path)
segpath = os.path.join(dataroot, seg_path)

# colimg = os.listdir(colpath)
# segimg = os.listdir(segpath)

X_train = os.listdir(os.path.join(colpath,'train'))
Y_train = os.listdir(os.path.join(segpath,'train'))
X_val = os.listdir(os.path.join(colpath,'val'))
Y_val = os.listdir(os.path.join(segpath,'val'))
X_test = os.listdir(os.path.join(colpath,'test'))
Y_test = os.listdir(os.path.join(segpath,'test'))


                
# X_train, X_test, Y_train, Y_test = train_test_split(colimg, segimg, random_state=123)

train_dataset = MyDataset(X_train, Y_train, dataroot, in_transforms = input_transforms, size = img_size,
	phase = 'train')
test_dataset = MyDataset(X_test, Y_test, dataroot, in_transforms = input_transforms, size = img_size,
	phase = 'test')
val_dataset = MyDataset(X_val, Y_val, dataroot, in_transforms = input_transforms, size = img_size,
	phase = 'val')


train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch, shuffle = True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset,batch_size=batch,shuffle=True)

In [16]:
rand_img = Image.open(os.path.join(segpath,'train',Y_train[38]))
print(np.unique(rand_img))
print(rand_img.size)

[ 0  2  3  5  7 11 12 13 14 17 21 28 32 33 34]
(1920, 1080)


In [0]:
########### Criterion ###########
optimizer = optim.Adam([
        {'params': [param for name, param in model.named_parameters() if name[-4:] == 'bias'],
         'lr': 2 * lr},
        {'params': [param for name, param in model.named_parameters() if name[-4:] != 'bias'],
         'lr': lr, 'weight_decay': weight_decay}
], betas=(momentum, 0.999))

In [0]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

In [0]:
########### Begin Training ###########
epoch = 0
while(epoch < max_epoch):
    print('Epoch', epoch+1)
    start = time.time()
    f = open(log, 'a')
    running_loss = 0
    acc = []
    # for j in ['train', 'val']:
    for j in ['train']:
        dataloader = train_dataloader
#         if(j=='val'):
#             dataloader = val_dataloader
        for i,data in enumerate(dataloader):

#             model.train()
            print(type(data[0]))
            
            print(data[0].size())
            print(data[1].size())
#             out = torchvision.utils.make_grid(data[0])

#             imshow(data[1][0].squeeze_(0))
            
            img = data[0].float().to(device)
            seg = data[1].long().to(device)
            print(img.type())
            print(seg.type())
            print(np.unique(seg.cpu().numpy()))
            # print(img.shape)
            # print(seg.shape)

#             npseg = seg.cpu().numpy()
#             un, weigh = np.unique(npseg, return_counts = True)
#             print(un)
#             print(len(weigh))
#             if(len(weigh)==10):
#                 weigh = np.insert(weigh, 0, 1)
#                 total = np.sum(weigh)
#             weigh = torch.from_numpy(weigh).float() / float(total)
#             weigh = weigh.reciprocal()
#             weigh = weigh.to(device)

            # print(img.shape, seg.shape)

            optimizer.zero_grad()
            output = model(img)

            test = torch.max(output.data, 1)[1]
            test = test.long()
            cur_accuracy = checkAccuracy(test, seg, batch)
            print(test.size())
            print(seg.size())
#             test1 = test.unsqueeze(3)
            iou = checkiou(test, seg, batch)
            acc.append(cur_accuracy)
            
            seg.squeeze_(1)
            criterion = nn.NLLLoss().to(device)             #removed .cuda()
            
            print(F.log_softmax(output,dim=1).size())
            print(seg.size())
            print(criterion(F.log_softmax(output, dim = 1), seg))
            loss = criterion(F.log_softmax(output, dim = 1), seg) / img.shape[0]
            loss.backward()

            optimizer.step()
            
            running_loss +=loss.item()
            summary_writer.add_scalar("accuracy", cur_accuracy)
            summary_writer.add_scalar("loss", loss.item())
            summary_writer.add_scalar("iou", iou.item())
            print(iou.shape, iou.item())
            print('loss:{0:.4f}'.format(loss.item()), "Epoch:{0} Batch:{1}/{2} accuracy:{3:.5f} IOU:{4:.5f}".format(epoch, i, len(dataloader), cur_accuracy, iou.item()))
            # print(j,i)
            if i % 200 == 0 :
                # print(img.max(), img.min())
                # exit()
                save_image(img, OUTPUTS_DIR + '{}_{}_scan.png'.format(epoch, i))
                # print(torch.max(output, 1)[1].shape)
                # print(seg.type, seg)
                out_img = 1.0 - torch.max(output, 1)[1].cpu().float().unsqueeze(1) / 255.0
                out_img = out_img.repeat(1,3,1,1)
                out_img[:, 0, :, :] = out_img[:, 0, :, :]*0.2
                out_img[:, 1, :, :] = out_img[:, 1, :, :]*0.4
                out_img[:, 2, :, :] = out_img[:, 2, :, :]*0.6
                save_image(out_img, OUTPUTS_DIR + '{}_{}_out.png'.format(epoch, i))
                # print(out_img.max(), out_img.min, out_img.shape, out_img.type)
                # out_img = transform_batch(to_tensor, out_img)
                # print(seg.max(), seg.min())
                
                seg_img = 1.0 - seg.float().cpu().unsqueeze(1) / 255.0
                seg_img = seg_img.repeat(1,3,1,1)
                seg_img[:, 0, :, :] = seg_img[:, 0, :, :]*0.2
                seg_img[:, 1, :, :] = seg_img[:, 1, :, :]*0.4
                seg_img[:, 2, :, :] = seg_img[:, 2, :, :]*0.6

                # print(seg_img.max(), seg_img.min())
                # seg_img = transform_batch(to_tensor, seg_img)
                # print(seg_img)
                # exit()
                save_image(seg_img, OUTPUTS_DIR + '{}_{}_seg.png'.format(epoch, i))
                # import ipdb; ipdb.set_trace()
				
            if i % 316 == 0:
                model.eval()
                ious = []
                for k, data in enumerate(val_dataloader):
                    x = data[0]
                    y = data[1]
                    # print(x.shape)
                    # print(y.shape)
                    with torch.no_grad():
                        x = x.to(device)
                        y = y.long().to(device)
                        out = model(x)
                        out_labels = torch.max(out.data, 1)[1]
                        iou = checkiou(out_labels, y, batch)
                        ious.append(iou)
                        summary_writer.add_scalar('batch val iou', iou.item())
                        print('Val Batch:{0}/{1} IOU:{2:.4f}'.format(k, len(val_dataloader), iou.item()))
                avg_iou = np.average(ious)
                summary_writer.add_scalar('avg test iou', avg_iou)
                print('Epoch: {0} TEST AVG IOU: {1:.4f}'.format(i, avg_iou))





    if(epoch%save_after == 0):
        torch.save(model.state_dict(), save_file)

    acc = np.asarray(acc)
    acc = np.mean(acc)

    s = 'Time of complete of epoch '+str(epoch+1)+' = '+str(time.time() - start)
    print(s)
    f.write(s+"\n")
    s = 'Loss after this epoch = '+str(running_loss)+' and average accuracy = '+str(acc)
    f.write(s+"\n")
    print(s)
    epoch+=1
    f.close()



Epoch 1
<class 'torch.Tensor'>
torch.Size([16, 3, 128, 128])
torch.Size([16, 1, 128, 128])
torch.cuda.FloatTensor
torch.cuda.LongTensor
[ 0  1  2  3  4  5  7  8 10 11 12 13 14 15 16 17 19 21 22 25 28 30 31 32
 33 34 35 36 37]
torch.Size([16, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 39, 128, 128])
torch.Size([16, 128, 128])
tensor(1.8453, device='cuda:0')
torch.Size([]) 0.43316543102264404
loss:0.1153 Epoch:0 Batch:0/316 accuracy:7.71759 IOU:0.43317
Val Batch:0/80 IOU:0.4336
Val Batch:1/80 IOU:0.4385
Val Batch:2/80 IOU:0.4493
Val Batch:3/80 IOU:0.4893
Val Batch:4/80 IOU:0.4253
Val Batch:5/80 IOU:0.4392
Val Batch:6/80 IOU:0.4602
Val Batch:7/80 IOU:0.5205
Val Batch:8/80 IOU:0.4592
Val Batch:9/80 IOU:0.4363
Val Batch:10/80 IOU:0.4321
Val Batch:11/80 IOU:0.4236
Val Batch:12/80 IOU:0.4040
Val Batch:13/80 IOU:0.4742
Val Batch:14/80 IOU:0.3810
Val Batch:15/80 IOU:0.4614
Val Batch:16/80 IOU:0.4425
Val Batch:17/80 IOU:0.5125
Val Batch:18/80 IOU:0.4264
Val Batch:19/80 IOU:0.4534
V